In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.10/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import snowflake.connector
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [3]:
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = 'SF_WH_CASE1'
database = 'SF_DB_CASE1'
schema = 'SF_CASE1'
role = 'SYSADMIN'  # Replace with your full access role

# Configure SQLAlchemy engine for Snowflake
engine = create_engine(URL(
    account=account,
    user=user,
    password=password,
    database=database,
    schema=schema,
    warehouse=warehouse,
    role=role,
))

# Ensure the Snowflake setup is completed successfully.
# This part can be moved to a separate setup script if it doesn't need to run every time.
with engine.connect() as conn:
    conn.execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")
    conn.execute("USE DATABASE SF_DB_CASE1")
    conn.execute("""
        CREATE WAREHOUSE IF NOT EXISTS SF_WH_CASE1
        WITH WAREHOUSE_SIZE = 'MEDIUM'
        WAREHOUSE_TYPE = 'STANDARD'
        AUTO_SUSPEND = 300
        AUTO_RESUME = TRUE;
    """)
    conn.execute("CREATE SCHEMA IF NOT EXISTS SF_CASE1")
    conn.execute("USE SCHEMA SF_CASE1")
    conn.execute("""
        CREATE OR REPLACE TABLE SUMMARY_FINAL (
            NAME_OF_THE_TOPIC VARCHAR(500),
            YEAR VARCHAR(255),
            LEVEL VARCHAR(255),
            INTRODUCTION_SUMMARY VARCHAR(10000),
            LEARNING_OUTCOMES VARCHAR(10000),
            LINK_TO_THE_SUMMARY_PAGE VARCHAR(1000),
            LINK_TO_THE_PDF_FILE VARCHAR(1000)
        );
    """)
    print("Snowflake setup completed successfully.")

# Load CSV data into DataFrame
csv_file_path = '../Webscrape/extracted_updated.csv'  # Adjust this path as needed
df = pd.read_csv(csv_file_path)

# Prepare DataFrame for Snowflake
df.columns = [col.upper() for col in df.columns]

# Transfer data from CSV to Snowflake
table_name = 'SUMMARY_FINAL'
try:
    df.to_sql(name=table_name, con=engine, index=False, if_exists='append', method='multi')
    print("Data transfer to Snowflake completed successfully.")
except Exception as e:
    print(f"Failed to transfer data to Snowflake: {e}")


/var/folders/dj/wcp4hvwd7wzglrm85gjck8lr0000gn/T/ipykernel_32194/519212564.py:26: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")


Snowflake setup completed successfully.
Failed to transfer data to Snowflake: 'Engine' object has no attribute 'cursor'


/var/folders/dj/wcp4hvwd7wzglrm85gjck8lr0000gn/T/ipykernel_32194/519212564.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(name=table_name, con=engine, index=False, if_exists='append', method='multi')
